In [59]:
import numpy as np
import pandas as pd

import sqlalchemy

import requests
import bs4
from glob import glob
import youtube_dl
import librosa

%load_ext autoreload
%autoreload 2

## Broad overview
For each genre
1. Get urls (_beatifulSoup_)
For each url
 1. Download - url to m4a(_youtube_dl_)
 3. MIR - m4a to sonic features(_librosa_)
 5. Remove *.m4a

## Detailed
for each genre:
1. collect urls (_beatifulSoup_)  
for each url
 1. __Download__ - url to m4a(_youtube_dl_)
 2. __Assign filepath__(_glob_)
 3. __MIR__ - m4a to array(_librosa_)
 4. __MIR__ - array to individual segments  
 5. __For each segment__
    1. __MIR__ - array to beats, mfccs, zcr...  
    2. Append to __db__(_sql/csv/json??_)
    
 5. Remove *.m4a

EDGE CASE - error handling

In [62]:
genre = 'country'

In [63]:
ydl_opts = {
    'format': 'bestaudio/best',
    'ignoreerrors': True,
    'outtmpl': '%(title)s.%(ext)s',
    'playlistend': 1,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
        'preferredquality': '192',
    }],
}
ydl = youtube_dl.YoutubeDL(ydl_opts)

In [64]:
response = requests.get('https://www.youtube.com/playlist?list=PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S')

In [65]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')

In [66]:
link_sfxs = ([a['href'] for a in soup.find_all('a', class_='pl-video-title-link')])

In [67]:
url = 'https://www.youtube.com' + link_sfxs[0]
ydl.download([url])

[youtube:playlist] Downloading playlist PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S - add --no-playlist to just download video ZEWGyyLiqY4
[youtube:playlist] PL3oW2tjiIxvQW6c-4Iry8Bpp3QId40S5S: Downloading webpage
[download] Downloading playlist: Best Country Songs of All Time - Top Country Music Videos
[youtube:playlist] playlist Best Country Songs of All Time - Top Country Music Videos: Downloading 1 videos
[download] Downloading video 1 of 1
[youtube] ZEWGyyLiqY4: Downloading webpage
[youtube] ZEWGyyLiqY4: Downloading video info webpage
[download] Destination: Blake Shelton - God's Country (Official Music Video).webm
[download] 100% of 3.29MiB in 00:01
[ffmpeg] Destination: Blake Shelton - God's Country (Official Music Video).m4a
Deleting original file Blake Shelton - God's Country (Official Music Video).webm (pass -k to keep)
[download] Finished downloading playlist: Best Country Songs of All Time - Top Country Music Videos


0

In [78]:
fp = glob('*.m4a')[0]

In [83]:
from genres import listen

In [80]:
features = listen(
    fp=fp,
    genre=genre
)

In [81]:
pd.DataFrame(features).head(2)

,song,genre,tempo,beats,chroma_stft,rmse,spec_cent,spec_bw,rolloff,zcr,...,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12
0,Blake Shelton - God's Country (Official Music ...,country,140.625,16,0.403930,0.143867,1507.265314,2044.584088,2611.347368,0.024373,...,17.539587,16.890709,-4.263755,5.052443,-2.659404,1.152874,-3.776567,1.217376,-0.12355,1.816821
1,Blake Shelton - God's Country (Official Music ...,country,93.750,13,0.512143,0.219117,1153.921325,2121.553618,2114.339594,0.015499,...,15.839293,21.270155,-4.460384,17.987693,1.827632,4.848183,4.518678,14.389262,6.49467,8.106324


In [82]:
! rm *.m4a